In [1]:
import numpy as np
import h5py


def load_stage2_with_shift(stage2_file, tsrc_list=None):
    """
    Returns:
        C_cfg shape (ncfg, nops, nops, Nt)
    """

    with h5py.File(stage2_file, "r") as f:
        C = f["Cij"][:]  # (ncfg, Ntsrc, nops, nops, Nt)

    ncfg, Ntsrc, nops, _, Nt = C.shape

    # If tsrc_list not given, assume evenly spaced
    if tsrc_list is None:
        tsrc_list = np.arange(Ntsrc) * (Nt // Ntsrc)

    C_cfg = np.zeros((ncfg, nops, nops, Nt), dtype=C.dtype)

    for c in range(ncfg):
        accum = np.zeros((nops, nops, Nt), dtype=C.dtype)

        for s, tsrc in enumerate(tsrc_list):

            data = C[c, s]  # (nops, nops, Nt)

            # shift so source is at t=0
            shifted = np.roll(data, -tsrc, axis=-1)

            accum += shifted

        C_cfg[c] = accum / Ntsrc

    return C_cfg

In [ ]:
C_cfg = load_stage2_with_shift("Cij_pseudoscalar_A1u_light.h5")

array([[[[-1.55440785e+00-4.60356038e-03j,
          -9.36898433e-03+7.78182773e-06j,
          -6.40808486e-03-5.59745593e-04j, ...,
          -1.17529921e-03-2.14382603e-04j,
          -3.31689551e-03-1.09409469e-03j,
          -1.97490210e-02+6.67825231e-04j],
         [-6.35417072e-05-2.51159171e-02j,
          -9.46836474e-04-1.93468621e-03j,
          -1.35018603e-03+1.27551968e-03j, ...,
           3.22492413e-04-3.15255356e-04j,
          -2.52880589e-05+9.56383423e-04j,
          -1.26589487e-04-2.13894180e-03j],
         [-4.42497361e-03-4.23625106e-03j,
           1.00739814e-03+1.54099173e-03j,
          -2.28827122e-04+3.88164010e-03j, ...,
           3.53354620e-04+4.56817342e-04j,
           1.12494566e-04+8.05327025e-04j,
           3.09842904e-04+1.29565396e-03j],
         [ 2.51279836e-03+1.94480939e-02j,
           2.23924995e-02+8.78951566e-03j,
           1.13933984e-02-2.13887326e-03j, ...,
          -4.72471303e-03-5.60238420e-04j,
          -6.37728435e-03-5.616

In [ ]:
Cavg = C_cfg.mean(axis=0)

import matplotlib.pyplot as plt
plt.plot(np.log(Cavg[0,0,:-1] / Cavg[0,0,1:]))
plt.show()